Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [42]:
import requests

server = "http://192.168.1.67/healthconnect/cdr/fhir/r4/"
api_url = server + "metadata"
response = requests.get(api_url)
#response.json()

Find a patient. Simple search for a patient named wrexham

In [43]:
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os
fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")

api_url = server + "Patient?name=wrexham"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': 'f3a46a8f-b2f7-4329-8d52-b57b47b3ee1d',
 'type': 'searchset',
 'timestamp': '2025-12-22T12:39:19Z',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient?name=wrexham'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient/18',
   'resource': {'resourceType': 'Patient',
    'address': [{'id': 'qEFQG',
      'line': ['30 PASTURE ROAD', 'BARTON-UPON-HUMBER', 'S HUMBERSIDE'],
      'period': {'start': '2025-06-24'},
      'postalCode': 'DN18 5HU',
      'use': 'home'}],
    'birthDate': '1987-02-18',
    'gender': 'unknown',
    'generalPractitioner': [{'id': 'ZGilw',
      'identifier': {'period': {'start': '2025-03-02'},
       'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
       'value': 'A20047'},
      'reference': 'Organization/1',
      'type': 'Organization'}],
    'id': '18',
    'identifier': [{'system': 'https://fhir.nhs.uk/Id/nhs-number',
      'type':

Now lets find diagnostic reports for this patient.

In [44]:
api_url = server + "DiagnosticReport?patient=18"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': 'bdd7c473-f993-4932-9b91-aec3c00b618e',
 'type': 'searchset',
 'timestamp': '2025-12-22T12:39:19Z',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport?patient=18'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport/22',
   'resource': {'resourceType': 'DiagnosticReport',
    'basedOn': [{'identifier': {'assigner': {'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
         'value': 'RBS'}},
       'system': 'https://nw-gmsa.github.io/Id/PLAC',
       'type': {'coding': [{'code': 'PLAC',
          'system': 'http://terminology.hl7.org/CodeSystem/v2-0203'}]},
       'value': '1000152867'},
      'type': 'ServiceRequest'}],
    'category': [{'coding': [{'code': 'GE',
        'system': 'http://terminology.hl7.org/CodeSystem/v2-0074'}]}],
    'code': {'coding': [{'code': 'R125_Cardiology',
       'display': 'PANEL: R125.1, R125.2 -

Can also return details about the organisation that created this.

In [45]:
api_url = server + "Organization/5"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Organization',
 'active': True,
 'id': '5',
 'identifier': [{'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': '699X0'}],
 'name': 'NORTH WEST GLH LED BY MANCHESTER UNIVERSITY NHS FOUNDATION TRUST',
 'partOf': {'display': 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST',
  'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': 'R0A'},
  'reference': 'Organization/4'},
 'type': [{'coding': [{'code': 'RO173',
     'display': 'PATHOLOGY LAB',
     'system': 'https://digital.nhs.uk/services/organisation-data-service/CodeSystem/ODSOrganisationRole'}]}],
 'meta': {'lastUpdated': '2025-12-19T11:20:46Z', 'versionId': '2'}}

Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [46]:
import fhirclient.models.diagnosticreport as dr
api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
json = response.json()

print(json['total'])

for entry in json['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    for coding in report.code.coding:
        print(coding.display)

6
DiagnosticReport 2025-10-14T15:59:16+00:00
PANEL: R125.1, R125.2 - Thoracic aortic aneurysm and dissection v1.2
DiagnosticReport 2025-10-14T15:59:16+00:00
PANEL: R125.1, R125.2 - Thoracic aortic aneurysm and dissection v1.2
DiagnosticReport 2025-10-14T15:59:16+00:00
PANEL: R125.1, R125.2 - Thoracic aortic aneurysm and dissection v1.2
DiagnosticReport 2025-12-17T12:22:52+00:00
TEST: Sanger Sequencing - Family follow-up testing - Rare Disease
TEST: Sanger Sequencing - Family follow-up testing - Rare Disease
DiagnosticReport 2025-12-17T12:25:36+00:00
PANEL: M80_AML_GX
Acute Myeloid Leukaemia, Multi-target NGS panel - small variant (NPM1, CEBPA, RUNX1, FLT3, IDH1, IDH2, KIT, WT1, ASXL1, SRSF2, STAG2, RAD21, TP53, KRAS, NRAS, KMT2A(MLL)-PTD), PPM1D, DDX41, PHF6, CUX1
DiagnosticReport 2025-12-18T12:14:40+00:00
TEST: PCR - FLT3 ITD mutation screen
Acute Myeloid Leukaemia, FLT3 ITD
Acute Myeloid Leukaemia, FLT3 TKD hotspot
